Calculate the magnitude of external input on the model, i.e. $Cs(t)$, and compare this with the magnitude of EMAs

In [1]:
import sys
sys.path.append('..')

import os
import matplotlib.pyplot as plt
import numpy as np
import torch as tc
import eval_reallabor_utils
import custom_rcparams as crc
import glob
from tqdm import tqdm

main_dir = r'/home/janik.fechtelpeter/Documents/bptt/results/Reallabor1.0CompleteSampleWithInputs'
results_dir = os.path.join(main_dir, '00_summary', 'impact_of_inputs')
eval_file = 'evaluation.csv'
metric = 'mae'
save_as = None
include_hyper = {}
exclude_hyper = {'feature': ['EMA_emotion_control', 'EMA_emotion_change'],
                 'participant': [38, 42]}#,
                #  'gt_var': 0}

color_version = 'construct'
sort_by = ['mae']

# results = eval_reallabor_utils.read_eval_csv(main_dir, eval_file)
# results = eval_reallabor_utils.include_exclude_hypers(results, include_hyper, exclude_hyper)

os.makedirs(results_dir, exist_ok=True)

model_dirs = eval_reallabor_utils.get_model_folders(main_dir)
model_dirs = [d for d in model_dirs if (('001' in d) and ('-6' in d))]

for dir in tqdm(model_dirs):

    model, test_dataset = eval_reallabor_utils.load_model_and_data(dir, 'latest')
    participant = model.args['participant']
    emas = model.dataset.timeseries['emas'].data - 3.5
    inputs = model.dataset.timeseries['inputs'].data
    inputs = tc.nan_to_num(inputs, nan=0)
    C = model.latent_model.latent_step.C.detach()
    impact_of_inputs = tc.einsum('zs,ts->tz',C,inputs)

    max_val = 7
    min_val = 1

    fig, axes = plt.subplots(2,1)
    im = axes[0].imshow(emas.T, vmin=min_val, vmax=max_val, cmap='coolwarm')
    axes[0].set_title('EMAs')
    plt.colorbar(im)
    axes[1].imshow(impact_of_inputs.T, vmin=min_val, vmax=max_val, cmap='coolwarm')
    axes[1].set_title('Impact of inputs')
    
    plt.tight_layout()

    plt.savefig(os.path.join(results_dir, f'{participant}.png'))
    

d:\ZI Mannheim\KI Reallabor\bptt\.venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
0it [00:00, ?it/s]
